# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import csv

with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

In [7]:
df = pd.read_csv("checkins.csv")
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [8]:
df.shape

(396634, 6)

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках, кластеризация будет работать долго (порядка часа). Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.

In [6]:
from sklearn.cluster import MeanShift

In [12]:
data = df[['latitude', 'longitude']]
data_test= data.iloc[:100000, :]
data_test.head()

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


In [13]:
data_test.shape

(100000, 2)

In [16]:
%%time
clst = MeanShift(bandwidth = 0.1)
clst.fit(data_test)

Wall time: 20min 56s


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [17]:
data_test['cluster'] = clst.predict(data_test)

C:\Users\alina\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
data_test.head()

,latitude,longitude,cluster
0,38.895112,-77.036366,5
1,33.800745,-84.410520,7
2,45.523452,-122.676207,30
3,40.764462,-111.904565,65
4,33.448377,-112.074037,1


In [29]:
clusters = pd.DataFrame(data_test.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
clusters.columns = ['clust_size']

cluster_centers = pd.DataFrame(clst.cluster_centers_)
cluster_centers.columns = ['cent_latitude', 'cent_longitude']

clusters_df = cluster_centers.join(clusters)
print clusters_df.shape
clusters_df.head()

(3231, 3)


,cent_latitude,cent_longitude,clust_size
0,40.717716,-73.991835,12506
1,33.449438,-112.002140,4692
2,33.446380,-111.901888,3994
3,41.878244,-87.629843,3363
4,37.688682,-122.409330,3526


In [30]:
clusters_df = clusters_df[clusters_df.clust_size > 15]
print clusters_df.shape
clusters_df.head()

(592, 3)


,cent_latitude,cent_longitude,clust_size
0,40.717716,-73.991835,12506
1,33.449438,-112.002140,4692
2,33.446380,-111.901888,3994
3,41.878244,-87.629843,3363
4,37.688682,-122.409330,3526


In [31]:
clusters_df.to_csv('clusters.csv', index = None)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [32]:
def get_distance (lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

offices = [ (33.751277, -118.188740),
            (25.867736, -80.324116),
            (51.503016, -0.075479),
            (52.378894, 4.885084),
            (39.366487, 117.036146),
            (-33.868457, 151.205134) ]

def get_min_dist(lat, lon):
    min_dist = None
    for (office_lat, office_lon) in offices:
        dist = get_distance(lat, lon, office_lat, office_lon)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist

In [34]:
clusters_df['min_distance'] = map(get_min_dist, clusters_df.cent_latitude, clusters_df.cent_longitude)

clusters_df.head()

,cent_latitude,cent_longitude,clust_size,min_distance
0,40.717716,-73.991835,12506,16.143720
1,33.449438,-112.002140,4692,6.193959
2,33.446380,-111.901888,3994,6.294241
3,41.878244,-87.629843,3363,17.598580
4,37.688682,-122.409330,3526,5.772048


In [35]:
clusters_df.sort_values('min_distance')

,cent_latitude,cent_longitude,clust_size,min_distance
420,-33.860630,151.204776,28,0.007835
370,52.372964,4.892317,31,0.009353
419,25.845672,-80.318891,28,0.022674
58,51.502991,-0.125537,254,0.050058
51,33.809878,-118.148924,281,0.070848
29,25.785812,-80.217938,564,0.134109
167,25.705350,-80.283429,80,0.167406
92,26.010098,-80.199991,138,0.188876
87,33.888325,-118.048928,100,0.195779
42,33.872986,-118.362091,384,0.211811


In [41]:
the_nearest_clusters = clusters_df.sort_values('min_distance')[:20]

the_nearest_clusters

,cent_latitude,cent_longitude,clust_size,min_distance
420,-33.860630,151.204776,28,0.007835
370,52.372964,4.892317,31,0.009353
419,25.845672,-80.318891,28,0.022674
58,51.502991,-0.125537,254,0.050058
51,33.809878,-118.148924,281,0.070848
29,25.785812,-80.217938,564,0.134109
167,25.705350,-80.283429,80,0.167406
92,26.010098,-80.199991,138,0.188876
87,33.888325,-118.048928,100,0.195779
42,33.872986,-118.362091,384,0.211811
